In [1]:
import numpy as np
from numpy import r_, c_ # Convenient syntax to create numpy arrays with e.g. r_[1,2,3] (concatenates in a row) or c_[1,2,3] (concatenates in a column)
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [2]:
# We'll be using these scikit-learn modules
from sklearn import preprocessing
from sklearn import metrics
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
# We'll import the models themselves as and when we use them

# Import census data
Load the CSVs as Pandas DataFrames, and the columns into a list.

In [3]:
# Parse metadata file to create list of column names
columns = []
for line in open('us_census_full/census_income_metadata.txt','r'):
    li=line.strip() # Strip trailing whitespace
    if not (li.startswith("|") or li.startswith("-") or li==''): # Strip comment lines and empty lines
        variable = li.split(':') # Split column description at colon
        columns.append(variable[0]) # Add column name to list

# Add final (problem) column name
columns.append('Income over 50k')

In [4]:
# Load US census data for training and validation
# Separator is a comma followed by a space
# NaN values are specified by '?'
census_learn = pd.read_csv('us_census_full/census_income_learn.csv', names = columns, sep=',\s', na_values=['?'], engine='python')
census_learn['Income over 50k'] = census_learn['Income over 50k']=='50000+.' # Convert last (problem) column to bool
census_learn.drop('instance weight', axis=1, inplace=True) # Drop instance weight column as not to be used in classifier
#census.head()

In [5]:
# Load US census data for test
# Separator is a comma followed by a space
# NaN values are specified by '?'
census_test = pd.read_csv('us_census_full/census_income_test.csv', names = columns, sep=',\s', na_values=['?'], engine='python')
census_test['Income over 50k'] = census_test['Income over 50k']=='50000+.' # Convert last (problem) column to bool
census_test.drop('instance weight', axis=1, inplace=True) # Drop instance weight column as not to be used in classifier
#census_test.head()

# Set up and transform features

First let's split off the income column into another dataset y to keep it separate from the variables X

In [6]:
census = census_learn
income = census['Income over 50k']
census = census.drop('Income over 50k', axis=1) # Drop income data

Census data contains missing values. As a basic method of dealing with these we'll drop them. An alternative would be to impute (i.e. infer from other data) their values.

In [7]:
columns_with_no_missing_data = census.notna().all().tolist() # Keep for later test data
census = census.loc[:,columns_with_no_missing_data]

Let's remove some features that are unlikely to be correlated to income

In [8]:
census = census.drop(['year'],axis=1) # The year the census entry was completed

Let's try adding some features

In [9]:
highly_educated_degrees = ['Bachelors degree(BA AB BS)','Masters degree(MA MS MEng MEd MSW MBA)','Associates degree-academic program','Associates degree-occup /vocational','Prof school degree (MD DDS DVM LLB JD)','Doctorate degree(PhD EdD)']
census['highly educated'] = census['education'].isin(highly_educated_degrees).astype(int) # Does this individual have a Bachelors degree or higher?

In this dataset, *sex* is either Male or Female. To avoid creating two colinear columns when we transform the data, we'll instead change this to "is male".

In [10]:
census['is male'] = (census['sex'] == 'Male').astype(int)
census = census.drop('sex',axis=1)

We have a lot of categorical data in our census. We can preprocess these into numerical values so logistic regression can function. We'll use a one-hot encoder so that each category becomes a feature on its own, to avoid ordering features in a manner that is unphysical.

In order to understand the model, we want to be able to identify the most important features.

To do this, we must encode the categorical and continuous features separately so that we can call categoricalTransformer.get_feature_names() as this method does not support passthrough. The two feature sets can then be combined.

First transform the categorical features using a one-hot encoder

In [11]:
# Identify categorical features, which will be tranformed into numerical values
categorical_feature_mask = census.dtypes==object
categorical_columns = census.columns[categorical_feature_mask].tolist() # List of categorical columns
# Add to the list some columns that are also categorical, despite their values being numeric
categorical_columns.extend(['detailed industry recode','detailed occupation recode','own business or self employed']) 

# Fit a One-Hot Encoder to each categorical column
# This turns each category of the categorical features into its very own feature
# Which prevents adding an unphysical ordering to features as an ordinal encoder would
categoricalTransformer = ColumnTransformer([('encoder', preprocessing.OneHotEncoder(), categorical_columns)], remainder='drop')
categoricalTransformer.fit(census) # Calculate the transformation
categorical_feature_names = categoricalTransformer.get_feature_names() # Follow the feature names through the transformation

Then pass through the numerical features unmodified

In [12]:
# Identify numerical features, which we will pass straight through the transformation
noncategorical_feature_mask = census.dtypes!=object
noncategorical_columns = census.columns[noncategorical_feature_mask].tolist() # List of noncategorical (continuous) columns
# Remove columns that are are categorical despite containing numbers
noncategorical_columns.remove('detailed industry recode');noncategorical_columns.remove('detailed occupation recode');noncategorical_columns.remove('own business or self employed') 

# Pass the continuous features through the transformation
noncategoricalTransformer = ColumnTransformer([('encoder', 'passthrough', noncategorical_columns)], remainder='drop')
noncategoricalTransformer.fit(census) # Calculate the transformation
noncategorical_feature_names = noncategorical_columns

Combine these two transformations to create the transformed feature data X and the target data y. Also keep track of the feature names for later analysis.

In [13]:
# Apply the transformation to the features
X_categorical = categoricalTransformer.transform(census).toarray()
X_noncategorical = noncategoricalTransformer.transform(census)
X = np.c_[X_categorical,X_noncategorical] # Combine the two feature sets columnwise
y = income

# Combine the feature names for later analysis
feature_names = r_[categorical_feature_names + noncategorical_feature_names]
census_transformed = pd.DataFrame(X,columns=feature_names)

Rescale data (zero mean and unit variance)

In [14]:
scaleTransformer = preprocessing.StandardScaler()
scaleTransformer.fit(X)
X = scaleTransformer.transform(X)

Separate the learing data into a training set (70%) and a cross-validation set (30%) so the accuracy of each model can be determined.

In [15]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=8888)

# Train model(s)

## Logistic regression

Train logistic regression

There are several options and hyperparameters we can play with here. In the first instance let's try scikit-learn's defaults.

In [16]:
from sklearn.linear_model import LogisticRegression

# Create and fit logistic regression model
log_reg = LogisticRegression(max_iter=500) # Optional: class_weight = 'balanced'
log_reg.fit(X_train,y_train)

# Score using the training data
training_score = log_reg.score(X_train,y_train)
print('Training score = {:2.2f}%'.format(training_score*100))

Training score = 95.31%


Check performance with cross-validation data

In [17]:
score = log_reg.score(X_val,y_val)
confusion = metrics.confusion_matrix(y_val,log_reg.predict(X_val))
precision = confusion[1,1] / (confusion[1,1] + confusion[0,1]) # TP / (TP + FP)
recall = confusion[1,1] / (confusion[1,1] + confusion[1,0]) # TP / (TP + FN)

print('Validation score = {:2.2f}%'.format(score*100))
print('       Precision = {:2.2f}%'.format(precision*100)) # Accuracy of positive predictions, i.e. how many of the predicted positives were accurate
print('          Recall = {:2.2f}%'.format(recall*100)) # True positive rate, i.e. how many of the true positives were found
print(confusion) # Confusion matrix [[TN,FP],[FN,TP]]

Validation score = 95.26%
       Precision = 73.82%
          Recall = 40.63%
[[55459   554]
 [ 2282  1562]]


We could try giving more weight to positive training data, which is sometimes beneficial if the set of positive results is small (as it is here)

In [18]:
# Create and fit logistic regression model
log_reg_balanced = LogisticRegression(max_iter=500, class_weight = 'balanced')
log_reg_balanced.fit(X_train,y_train)

# Score using the training data
training_score = log_reg_balanced.score(X_train,y_train)
print('Training score = {:2.2f}%'.format(training_score*100))

Training score = 85.50%


Check performance with cross-validation data

In [19]:
score = log_reg_balanced.score(X_val,y_val)
confusion = metrics.confusion_matrix(y_val,log_reg_balanced.predict(X_val))
precision = confusion[1,1] / (confusion[1,1] + confusion[0,1]) # TP / (TP + FP)
recall = confusion[1,1] / (confusion[1,1] + confusion[1,0]) # TP / (TP + FN)

print('Validation score = {:2.2f}%'.format(score*100))
print('       Precision = {:2.2f}%'.format(precision*100)) # Accuracy of positive predictions, i.e. how many of the predicted positives were accurate
print('          Recall = {:2.2f}%'.format(recall*100)) # True positive rate, i.e. how many of the true positives were found
print(confusion) # Confusion matrix [[TN,FP],[FN,TP]]

Validation score = 85.62%
       Precision = 29.60%
          Recall = 89.91%
[[47793  8220]
 [  388  3456]]


Balancing the classes resulted in fewer false negatives but far more false positives, with a lower overall score.

There are several other options and parameters I have tried modifying, including the solvers, and types and strengths of regularisation, but these are the best I got through logistic regression.

## Decision tree

In [20]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(max_depth=10, random_state=9354)
decision_tree.fit(X_train,y_train)

# Score using the training data
training_score = decision_tree.score(X_train,y_train)
print('Training score = {:2.2f}%'.format(training_score*100))

Training score = 95.75%


In [21]:
score = decision_tree.score(X_val,y_val)
confusion = metrics.confusion_matrix(y_val,decision_tree.predict(X_val))
precision = confusion[1,1] / (confusion[1,1] + confusion[0,1]) # TP / (TP + FP)
recall = confusion[1,1] / (confusion[1,1] + confusion[1,0]) # TP / (TP + FN)

print('Validation score = {:2.2f}%'.format(score*100))
print('       Precision = {:2.2f}%'.format(precision*100)) # Accuracy of positive predictions
print('          Recall = {:2.2f}%'.format(recall*100)) # True positive rate
print(confusion) # Confusion matrix [[TN,FP],[FN,TP]]

Validation score = 95.12%
       Precision = 72.90%
          Recall = 38.22%
[[55467   546]
 [ 2375  1469]]


A single decision tree performed strictly worse than logistic regression.

## Random forest
A random forest classifier trains a number of decision trees with a random seed, and returns their average prediction (in the case of classification, the mode result).

In [22]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(random_state=1357, n_estimators=200, n_jobs=-1)
random_forest.fit(X_train,y_train)

# Score using the training data
training_score = random_forest.score(X_train,y_train)
print('Training score = {:2.2f}%'.format(training_score*100))

Training score = 99.94%


In [23]:
score = random_forest.score(X_val,y_val)
confusion = metrics.confusion_matrix(y_val,random_forest.predict(X_val))
precision = confusion[1,1] / (confusion[1,1] + confusion[0,1]) # TP / (TP + FP)
recall = confusion[1,1] / (confusion[1,1] + confusion[1,0]) # TP / (TP + FN)

print('Validation score = {:2.2f}%'.format(score*100))
print('       Precision = {:2.2f}%'.format(precision*100)) # Accuracy of positive predictions
print('          Recall = {:2.2f}%'.format(recall*100)) # True positive rate
print(confusion) # Confusion matrix [[TN,FP],[FN,TP]]

Validation score = 95.37%
       Precision = 76.02%
          Recall = 40.82%
[[55518   495]
 [ 2275  1569]]


This random forest gives slightly better false positive and false negative rates than logistic regression, and a slightly higher overall score.

**We'll use this as our final model.**

#### Other classifiers
Scikit-learn offers other classification models, of which I also tried an AdaBoost classifier, Multi-layer Perceptron classifier, and Quadratic discriminant analysis.
These performed worse than than logistic regression and random forest models with the default settings, and I did not spend the time attempting to optimise these.

# Test
Let's test our model's real performance with data we haven't trained or validated with

Apply same transformations as with training data. (Ideally this would be using a pipeline, but this turned out to be complicated - see discussion section)

In [24]:
census = census_test
income = census['Income over 50k']
census = census.drop('Income over 50k', axis=1) # Drop income data

In [25]:
census = census.loc[:,columns_with_no_missing_data] # Drop columns with missing data

In [26]:
census = census.drop(['year'],axis=1)

In [27]:
highly_educated_degrees = ['Bachelors degree(BA AB BS)','Masters degree(MA MS MEng MEd MSW MBA)','Associates degree-academic program','Associates degree-occup /vocational','Prof school degree (MD DDS DVM LLB JD)','Doctorate degree(PhD EdD)']
census['highly educated'] = census['education'].isin(highly_educated_degrees).astype(int)

In [28]:
census['is male'] = (census['sex'] == 'Male').astype(int)
census = census.drop('sex',axis=1)

In [29]:
# Apply the transformation to the features
X_categorical = categoricalTransformer.transform(census).toarray()
X_noncategorical = noncategoricalTransformer.transform(census)
X_test = np.c_[X_categorical,X_noncategorical] # Combine the two feature sets columnwise
y_test = income

X_test = scaleTransformer.transform(X_test)

Score the model on the test data. Best performing model according to cross-validation was the **random forest**.

In [30]:
final_model = random_forest

In [31]:
score = final_model.score(X_test,y_test)
confusion = metrics.confusion_matrix(y_test,final_model.predict(X_test))
precision = confusion[1,1] / (confusion[1,1] + confusion[0,1]) # TP / (TP + FP)
recall = confusion[1,1] / (confusion[1,1] + confusion[1,0]) # TP / (TP + FN)

print('Test score = {:2.2f}%'.format(score*100))
print(' Precision = {:2.2f}%'.format(precision*100)) # Accuracy of positive predictions
print('    Recall = {:2.2f}%'.format(recall*100)) # True positive rate
print(confusion) # Confusion matrix [[TN,FP],[FN,TP]]

Test score = 95.39%
 Precision = 73.49%
    Recall = 40.06%
[[92682   894]
 [ 3708  2478]]


# Insigts and analysis of model

From these models we can determine what features in the US census dataset are correlated with high-earners. 

To find out these features, we investigate which features the model deemed important, rank them by their coefficient, and lookup which features they correspond to.

In [32]:
important_features_arg = np.flipud(np.argsort(np.abs(final_model.feature_importances_))[-10:]) # 10 most important features by coefficient
important_features_coef = final_model.feature_importances_[important_features_arg] # Their coefficients
important_features = pd.DataFrame(c_[feature_names[important_features_arg],important_features_coef],columns=['feature','importance']) # Format it nicely
print('Features picked out by model {:s}'.format(str(final_model.__class__)))
important_features

Features picked out by model <class 'sklearn.ensemble._forest.RandomForestClassifier'>


,feature,importance
0,age,0.09885442762707386
1,dividends from stocks,0.07501369317765014
2,capital gains,0.06406771209696899
3,num persons worked for employer,0.043058810851889125
4,highly educated,0.03238181643999346
5,weeks worked in year,0.030622688671986146
6,is male,0.02629339848752002
7,capital losses,0.02133598669847576
8,encoder__x5_Executive admin and managerial,0.016356786012077237
9,encoder__x19_2,0.015280810431397734


The random forest picked out several important features. A quick summary:
1. High earners tend to be older (age)
2. Returns on investments are correlated with high earnings (dividends from stocks, capital gains)
3. High earners tend to work for very large companies (num persons worked for employer)
4. Being highly educated (at least a Bachelors degree) increases earnings (highly educated)
5. Working more weeks in a year is indicative of higher earning (weeks worked in year)
6. Male privilege is real (is male)
7. Executive positions are correlated with high earnings (Executive admin and managerial)

A downside of the analysis above is that the *importance* returned by a random forest does not indicate a positive or negative correlation, so some common sense was needed in interpreting its results.
The logistic regression has an advantage over the random forest for this, as its coefficients can be positive or negative:

In [33]:
important_features_arg = np.flipud(np.argsort(np.abs(log_reg.coef_[0]))[-10:]) # 10 most important features by coefficient
important_features_coef = log_reg.coef_[0,important_features_arg] # Their coefficients
important_features = pd.DataFrame(c_[feature_names[important_features_arg],important_features_coef],columns=['feature','coefficient']) # Format it nicely
print('Features picked out by model {:s}'.format(str(log_reg.__class__)))
important_features

Features picked out by model <class 'sklearn.linear_model._logistic.LogisticRegression'>


,feature,coefficient
0,weeks worked in year,0.896624387957522
1,age,0.7393690418987341
2,encoder__x10_Nonfiler,-0.5807120385187193
3,is male,0.5624864818998175
4,capital gains,0.4604130595247235
5,encoder__x10_Joint both under 65,0.4342619002298662
6,num persons worked for employer,0.38482260438650867
7,encoder__x15_Mother only present,-0.37962440121208363
8,dividends from stocks,0.34578452894087164
9,encoder__x1_Children,-0.34345597567682534


On the whole, the logistic regression has identified similar parameters to the random forest, but has some additional insights:
1. Filing a joint income tax with both filers under 65 is correlated with a high income.
2. Offspring of single mothers are disadvantaged
3. Being a child limits your earnings (unsurprisingly)

# Discussion

### What was done
We loaded the training data and transformed it into a state where a classifier can be trained on it. The solvers used here cannot handle missing data, so as a simple way to account for this the features with missing data were dropped. We removed the *year* feature as this refers to the year the census entry was completed and is unlikely to be predictive of an individual's earning power, and added an additional *highly educated* feature to identify individuals with Bachelors degrees or higher. We also transformed the Male/Female classification and replaced it with a *is male* feature, as otherwise we would have two important colinear features.

We applied a *one-hot encoder* to the categorical features of the data, making each possible category a feature in itself to prevent any unphysical ordering to data. Numerical features (e.g. *age*, *wage per hour*, *dividends from stocks*) are passed straight through. We then rescale the data o that each feature has zero mean and unit variance, which often assists with the convergence of the solvers.

The learning dataset was split 70%/30% into a training set and a cross-validation set, so that the performance of each solver could be quantified without simply using the training set (which makes the performance evaluation susceptible to overfitting) or using the test set, which should be reserved until the final model is ready.

Three classifiers were presented and trained above: a logistic regression; a decision tree; a random forest. The logistic regression and random forest performed very similarly, with the random forest just winning. 

The random forest was evaluated with the test set, with a score = 95.12%, precision = 72.90%, recall = 38.22%. 

Analysis of the random forest and linear regression models identified the typical profile of high earners in the US (detailed in the section above)

### What more could be done
Having tried several models, the best model had a score of 95.4%. This doesn't seem that much better than the 93.7% accuracy of a model which assumes nobody earns more than $50k in a year. This could be improved by providing more features, or by training on a much larger dataset. 

We handled missing data by simply dropping the features with NaN values. It is possible instead to impute the missing values, i.e. to infer them from the known part of the data. This could involve replacing NaN values with the mean (or mode) of that feature, or might use other features to estimate the missing value. I might have implemented this if I could afford to spend the time.

We identified Male/Female as colinear features that should be transformed into a single feature in the model. There are additional colinear (or highly correlated) features that could be removed. These can be identified using ``df.corr()`` and identifying pairs with values equal (or close) to 1.0. I implemented this but found it made effectively no difference to the model accuracy, so ultimately removed it from the transformation to keep it simple. Some machine learning methods fail with colinear features, for these such a transformation would be necessary. It is also important to account for colinearity for important features (as we did with *is male*) as it can affect the estimated coefficients.

Additional classifiers available in scikit-learn include AdaBoost classifier, Multi-layer Perceptron classifier, and Quadratic discriminant analysis. These were briefly tried (not shown), but not much time was spent trying to optimise these, primarily due to my being less familiar with these models.

### What I found challenging
This dataset is built significantly around categorical data, such as *education*, *tax filer stat*, or *citizenship*. These features had to be transformed into a meaningful numeric dataset which the models could be trained on, while leaving the noncategorical features unchanged. For this I had to apply a *one-hot encoder*, which I had heard of but not previously implemented.

I was hoping to turn this transformation into a pipeline so that the entire model (including transformation and estimation) could be encapsulated into one object. This was complicated by the need to maintain a correspondence between the input feature and resulting transformed feature in the model. Scikit-learn's ``get_feature_names()`` has not yet been implemented for passthrough models, so to implement this in a pipeline-friendly way would require a custom transformation I could not spend the time implementing.

I was also unsure of the best way to treat the *num persons worked for employer* feature. This is arguably categorical, as the census technical docs indicate it takes 6 possible values describing the total number of persons who work for their employer, e.g. (0 = nil, 1 = <10, 2 = 10-24, ... , 6 = 1000+). But it also seems inappropriate to treat it with a one-hot encoder, as this data is ordered despite being categorised. In the end I left it with the noncategorical features as it turned out to make very little difference to the models' accuracy.